# Pesquisa Operacional (_Operational Research_)

Pesquisa Operacional é uma área do conhecimento que utiliza métodos matemáticos, estatísticos e computacionais para apoiar a tomada de decisões em problemas complexos. Seu principal objetivo é encontrar soluções ótimas ou eficientes para situações que envolvem múltiplas variáveis e restrições, como alocação de recursos, logística, produção, planejamento e gerenciamento de sistemas. Por meio de modelos como programação linear, teoria das filas e simulação, a pesquisa operacional ajuda a melhorar processos e a aumentar a eficiência em diversos setores, como indústria, transporte, saúde e serviços.

## Cálculo de quantidade de papéis a serem adquiridos

A ideia do presente laboratório trabalho é realizar um cálculo de quantidade de papéis a serem adquiridos a partir de um montante inicial a ser investido e considerando apenas uma lista de ativos previamente avaliados e selecionados para composição de uma carteira teórica.

Para isso será utilizada técnica de Pesquisa Operacional (_Operational Research_ ou _OR_) e programação linear (_Linear Programming_ ou _LP_).

O método e as condições de contorno a serem utilizadas serão descritas ao longo do _notebook_.

> O nome dos papéis foram alterados para evitar quaisquer sugestões de investimento.

## Instalando a biblioteca `ortools` para utilização

Comando no `bash`:
```bash
pip install ortools
```

Opção para rodar diretamente a partir de uma célula do jupyter:
```python
!pip install ortools
```

## Importando o módulo `pywraplp` da biblioteca `ortools.linear_solver`

O módulo `pywraplp` é uma interface Python da OR-Tools. Ele permite modelar e resolver problemas de programação linear (PL) e programação inteira (PI) de forma eficiente. Com o `pywraplp`, é possível definir variáveis, restrições e funções objetivo de forma estruturada, utilizando diferentes solucionadores como o CBC, GLOP ou SCIP. É uma ferramenta poderosa e prática para aplicar conceitos de pesquisa operacional diretamente em projetos com Python.

O nome sugere `py` + `wrap` + `lp`, ou seja, um empacotamento em python de _Linear Programming_ (LP).

In [411]:
from ortools.linear_solver import pywraplp

## Inicializando o `solver`

O `CBC_MIXED_INTEGER_PROGRAMMING` é um solucionador disponível no OR-Tools (usado com `pywraplp`) que resolve problemas de programação inteira mista (_Mixed Integer Programming_ MIP). Ele é baseado no _**C**oIn-OR **B**ranch and **C**ut_ (CBC), um solucionador _open source_ rápido e eficiente, ideal para lidar com modelos que possuem variáveis inteiras e contínuas.

Ele faz parte do projeto COIN-OR (_Computational Infrastructure for Operations Research_), que reúne ferramentas livres para pesquisa operacional.

O CBC utiliza técnicas como _branch-and-bound_ e _cutting planes_ para encontrar soluções ótimas para problemas com variáveis inteiras. Ele é conhecido por ser:

- eficiente, mesmo em problemas grandes;

- flexível, podendo ser usado em várias linguagens (C++, Python, etc.);

- gratuito e _open source_, o que o torna uma boa alternativa a solucionadores comerciais como CPLEX e Gurobi.



In [412]:
# Cria o solucionador (solver)
solver = pywraplp.Solver(
    "Maximizar proventos", # Nome
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING, # Engine do solver
)

## Definindo variáveis de decisão

Aqui achei interessante utilizar o _ticker_ dos papéis como nome da variável para facilitar a referenciação. A variável com o nome do _ticker_ representará a quantidade do mesmo, que é a resposta que buscamos a fim de maximizar o valor final estimado de proventos mensais &ndash;  tomando como base o _Dividend Yeld_ (_DY_) médio dos últimos 12 meses, ciente que os resultados passados não são garantia para o futuro.

In [413]:
## Variáveis _min_qtd informa a quantidade mínima a ser considerada.
## É particularmente útil especialmente para informar como quantidade mínima
## a quantidade que o cliente já possui daquele determinado papel.

# quantidade mínima # ou quantidade de papéis que já o cliente possui
AAAA11_min_qtd = 0
BBBB11_min_qtd = 0
CCCC11_min_qtd = 0
DDDD11_min_qtd = 0
EEEE11_min_qtd = 0
FFFF11_min_qtd = 0
GGGG11_min_qtd = 0
HHHH11_min_qtd = 0
IIII11_min_qtd = 0

## Inicialização das variáveis definindo as primeiras condições de contorno,
## nesse caso, informando que a quantidade de papéis poderá assumir quaisquer
## entre dois valores especificados, nesse caso os valores são o mínimo para
## cada papel e o máximo está setado como infinito, pois utilizaremos outras
## regras para delimitar esse máximo.

# LLLL## = quantidade de papéis
# LLLL## = solver.IntVar( # IntVar define que será uma variável que apenas
#                           assumirá valores inteiros
#     LLLL11_min_qtd,     # Recebe o limite inferior para a variávl, neste caso
#                         # o que foi setado em LLLL11_min_qtd
#     solver.infinity(),  # Recebe o limite superior para a variávl, neste caso
#                         # solver.infinity() que representa o infinito
#     "LLLL11"            # Nomeia a variável
#     )

AAAA11 = solver.IntVar(AAAA11_min_qtd, solver.infinity(), "AAAA11")
BBBB11 = solver.IntVar(BBBB11_min_qtd, solver.infinity(), "BBBB11")
CCCC11 = solver.IntVar(CCCC11_min_qtd, solver.infinity(), "CCCC11")
DDDD11 = solver.IntVar(DDDD11_min_qtd, solver.infinity(), "DDDD11")
EEEE11 = solver.IntVar(EEEE11_min_qtd, solver.infinity(), "EEEE11")
FFFF11 = solver.IntVar(FFFF11_min_qtd, solver.infinity(), "FFFF11")
GGGG11 = solver.IntVar(GGGG11_min_qtd, solver.infinity(), "GGGG11")
HHHH11 = solver.IntVar(HHHH11_min_qtd, solver.infinity(), "HHHH11")
IIII11 = solver.IntVar(IIII11_min_qtd, solver.infinity(), "IIII11")

## Variáveis

In [414]:
## Variáveis de configuração

# valor máximo a ser considerado para o investimento / aporte
valor_maximo = 10_000.58 # curiosidade: pode-se utilizar o '_' como separador
                         # de milhares no python, para melhorar a legibilidade

# (segundo meus testes, com os valores atuais para quantidade, preço, valores e)
# (proventos, o modelo deste exemplo só está chegando em status OPTIMAL com)
# (valor_máximo >= 10_000.58)

# quantidade de papeis distintos a serem considerados
divisao = 9

# fator de variação que permite flexibilidade entre os valores máximos entre os
# montantes a serem alocados em cada um dos papéis
fator = 0.10 / 2  # +ou- 10%

## No caso deste laboratório, considera-se que o cliente pretende realizar uma
## alocação diversificada equilibrada entre os 9 tipos de ativos e considera
## ±10% de diferença entre as alocações

# manipulação de controle considerando a varição da flexibilidade
valor_equi_min = (valor_maximo / divisao) * (1 - fator) # p.ex.: 0.90
valor_equi_max = (valor_maximo / divisao) * (1 + fator) # p.ex.: 1.10

## Restrições

In [415]:
## Nas variáveis _prc informaremos ao algorítmo os valores unitários atualizados
## dos papéis selecionados para serem considerados nas condições de contorno

# preços
AAAA11_prc =  75.15 # era   89.58
BBBB11_prc = 150.90 # era  156.00
CCCC11_prc = 100.76 # era  104.97
DDDD11_prc = 145.84 # era  197.98
EEEE11_prc =  44.75 # era   50.29
FFFF11_prc =  90.31 # era   96.82
GGGG11_prc = 100.12 # era  103.57
HHHH11_prc =  67.70 # era   72.66
IIII11_prc =  92.89 # era   96.94

## As restrições abaixo informam ao algoritmo as condições de contorno
## superiores para cada alocação, limitando que o valor máximo a ser alocado em
## cada uma seja no máximo 5% acima do 9 avos do investimento total.

# condições de contorno superior para o total alocado em cada papel
solver.Add(AAAA11 * AAAA11_prc <= valor_equi_max)
solver.Add(BBBB11 * BBBB11_prc <= valor_equi_max)
solver.Add(CCCC11 * CCCC11_prc <= valor_equi_max)
solver.Add(DDDD11 * DDDD11_prc <= valor_equi_max)
solver.Add(EEEE11 * EEEE11_prc <= valor_equi_max)
solver.Add(FFFF11 * FFFF11_prc <= valor_equi_max)
solver.Add(GGGG11 * GGGG11_prc <= valor_equi_max)
solver.Add(HHHH11 * HHHH11_prc <= valor_equi_max)
solver.Add(IIII11 * IIII11_prc <= valor_equi_max)

## As restrições abaixo informam ao algoritmo as condições de contorno
## inferiores para cada alocação, limitando que o valor mínimo a ser alocado em
## cada uma seja no máximo 5% abaixo do 9 avos do investimento total.

# condições de contorno inferior para o total alocado em cada papel
solver.Add(AAAA11 * AAAA11_prc >= valor_equi_min)
solver.Add(BBBB11 * BBBB11_prc >= valor_equi_min)
solver.Add(CCCC11 * CCCC11_prc >= valor_equi_min)
solver.Add(DDDD11 * DDDD11_prc >= valor_equi_min)
solver.Add(EEEE11 * EEEE11_prc >= valor_equi_min)
solver.Add(FFFF11 * FFFF11_prc >= valor_equi_min)
solver.Add(GGGG11 * GGGG11_prc >= valor_equi_min)
solver.Add(HHHH11 * HHHH11_prc >= valor_equi_min)
solver.Add(IIII11 * IIII11_prc >= valor_equi_min)

## A condição de contorno abaixo indica que a quantidade a ser definida para
## recomendação de compra de cada papel multiplicada pelo preço informado de
## cada uma, quando somadas não ultrapassem o valor total que está sendo
## considerado para investimento/aporte.

# condições de contorno para o limite do investimento/aporte total
solver.Add(                \
     AAAA11 * AAAA11_prc + \
     BBBB11 * BBBB11_prc + \
     CCCC11 * CCCC11_prc + \
     DDDD11 * DDDD11_prc + \
     EEEE11 * EEEE11_prc + \
     FFFF11 * FFFF11_prc + \
     GGGG11 * GGGG11_prc + \
     HHHH11 * HHHH11_prc + \
     IIII11 * IIII11_prc <= valor_maximo
)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x1073ef480> >

In [416]:
## As variáveis _dy informam ao algoritmo quais são os valores esperados para os
## proventos a serem pagos por cada unidade de papel a que ele se relaciona de
## modo a permitir que seja calculada a função objetivo que é otimizar a
## quantidade de cada papel a ser adquirido para a formação da carteira, dentro
## dos limites de valores estipulados para investimento/aporte, de modo a
## maximizar os valores dos proventos a serem recebidos pela carteira definida

# valores esperados para os proventos a serem pagos por cada unidade de papel
AAAA11_dy = 0.80 # era 0.78
BBBB11_dy = 1.10 # era 1.10
CCCC11_dy = 0.85 # era 0.64
DDDD11_dy = 2.14 # era 2.25
EEEE11_dy = 0.41 # era 0.35
FFFF11_dy = 1.03 # era 0.90
GGGG11_dy = 0.80 # era 0.97
HHHH11_dy = 0.74 # era ?
IIII11_dy = 0.82 # era ?

## Definição da função obtetivo de forma que a somatória da quantidade de papeis
## multiplicada pelo valor estimado a ser pago em forma de proventos por cada
## um dos papeis seja maximizado.

# definição da função objetivo
solver.Maximize(
    AAAA11 * AAAA11_dy + \
    BBBB11 * BBBB11_dy + \
    CCCC11 * CCCC11_dy + \
    DDDD11 * DDDD11_dy + \
    EEEE11 * EEEE11_dy + \
    FFFF11 * FFFF11_dy + \
    GGGG11 * GGGG11_dy + \
    HHHH11 * HHHH11_dy + \
    IIII11 * IIII11_dy
)

In [417]:
## execução do solver (solucionador) e armazenamento do retorno da função na 
## variável status

status = solver.Solve()

```python
# CLASS VARIABLES for SOLUTION STATUS
pywraplp.Solver.OPTIMAL       = 0
pywraplp.Solver.FEASIBLE      = 1
pywraplp.Solver.INFEASIBLE    = 2
pywraplp.Solver.UNBOUNDED     = 3
pywraplp.Solver.ABNORMAL      = 4
pywraplp.Solver.MODEL_INVALID = 5
pywraplp.Solver.NOT_SOLVED    = 6
```

In [418]:
## Impressão dos resultados para facilitação da leitura e tomada de decisão

## Só executa-se a impressão se o status do solver for o OPTIMAL, ou seja,
## ótimo

# impressão dos resultados
if status == pywraplp.Solver.OPTIMAL: # resultado ótimo
# if status <= pywraplp.Solver.FEASIBLE: # resultado factícvel
    print(f"Solução:")
    print(f"Calculado em {solver.wall_time():.2f}ms utilizando {solver.iterations()} iterações")
    print('-' * 20)
    # armazena nas variáveis _total a quantidade total definida pela solução
    AAAA11_total = AAAA11.solution_value()
    BBBB11_total = BBBB11.solution_value()
    CCCC11_total = CCCC11.solution_value()
    DDDD11_total = DDDD11.solution_value()
    EEEE11_total = EEEE11.solution_value()
    FFFF11_total = FFFF11.solution_value()
    GGGG11_total = GGGG11.solution_value()
    HHHH11_total = HHHH11.solution_value()
    IIII11_total = IIII11.solution_value()

    # calcula quantos papeis devem ser comprados considerando-se o total do
    # resultado subtraídos pelos valores iniciais informados inicialemente
    # caso o cliente já possuísse quantidades de um determinado papel
    AAAA11_buy = AAAA11_total - AAAA11_min_qtd
    BBBB11_buy = BBBB11_total - BBBB11_min_qtd
    CCCC11_buy = CCCC11_total - CCCC11_min_qtd
    DDDD11_buy = DDDD11_total - DDDD11_min_qtd
    EEEE11_buy = EEEE11_total - EEEE11_min_qtd
    FFFF11_buy = FFFF11_total - FFFF11_min_qtd
    GGGG11_buy = GGGG11_total - GGGG11_min_qtd
    HHHH11_buy = HHHH11_total - HHHH11_min_qtd
    IIII11_buy = IIII11_total - IIII11_min_qtd

    # calcula valor a ser utilizado para aquisição de cada quantidade de papel
    AAAA11_amount = AAAA11_buy * AAAA11_prc
    BBBB11_amount = BBBB11_buy * BBBB11_prc
    CCCC11_amount = CCCC11_buy * CCCC11_prc
    DDDD11_amount = DDDD11_buy * DDDD11_prc
    EEEE11_amount = EEEE11_buy * EEEE11_prc
    FFFF11_amount = FFFF11_buy * FFFF11_prc
    GGGG11_amount = GGGG11_buy * GGGG11_prc
    HHHH11_amount = HHHH11_buy * HHHH11_prc
    IIII11_amount = IIII11_buy * IIII11_prc

    # calcula o valor total alocado em cada um dos papéis
    AAAA11_tot_amount = (AAAA11_min_qtd + AAAA11_buy) * AAAA11_prc
    BBBB11_tot_amount = (BBBB11_min_qtd + BBBB11_buy) * BBBB11_prc
    CCCC11_tot_amount = (CCCC11_min_qtd + CCCC11_buy) * CCCC11_prc
    DDDD11_tot_amount = (DDDD11_min_qtd + DDDD11_buy) * DDDD11_prc
    EEEE11_tot_amount = (EEEE11_min_qtd + EEEE11_buy) * EEEE11_prc
    FFFF11_tot_amount = (FFFF11_min_qtd + FFFF11_buy) * FFFF11_prc
    GGGG11_tot_amount = (GGGG11_min_qtd + GGGG11_buy) * GGGG11_prc
    HHHH11_tot_amount = (HHHH11_min_qtd + HHHH11_buy) * HHHH11_prc
    IIII11_tot_amount = (IIII11_min_qtd + IIII11_buy) * IIII11_prc

    # imprime a relação do resultado 
    print(f"AAAA11: {AAAA11_total} = {AAAA11_min_qtd} (now) + {AAAA11_buy} (buy) \t | + R$ {AAAA11_amount:.2f} | R$ {AAAA11_tot_amount:.2f}")
    print(f"BBBB11: {BBBB11_total} = {BBBB11_min_qtd} (now) + {BBBB11_buy} (buy) \t | + R$ {BBBB11_amount:.2f} | R$ {BBBB11_tot_amount:.2f}")
    print(f"CCCC11: {CCCC11_total} = {CCCC11_min_qtd} (now) + {CCCC11_buy} (buy) \t | + R$ {CCCC11_amount:.2f} | R$ {CCCC11_tot_amount:.2f}")
    print(f"DDDD11: {DDDD11_total} = {DDDD11_min_qtd} (now) + {DDDD11_buy} (buy) \t | + R$ {DDDD11_amount:.2f} | R$ {DDDD11_tot_amount:.2f}")
    print(f"EEEE11: {EEEE11_total} = {EEEE11_min_qtd} (now) + {EEEE11_buy} (buy) \t | + R$ {EEEE11_amount:.2f} | R$ {EEEE11_tot_amount:.2f}")
    print(f"FFFF11: {FFFF11_total} = {FFFF11_min_qtd} (now) + {FFFF11_buy} (buy) \t | + R$ {FFFF11_amount:.2f} | R$ {FFFF11_tot_amount:.2f}")
    print(f"GGGG11: {GGGG11_total} = {GGGG11_min_qtd} (now) + {GGGG11_buy} (buy) \t | + R$ {GGGG11_amount:.2f} | R$ {GGGG11_tot_amount:.2f}")
    print(f"HHHH11: {HHHH11_total} = {HHHH11_min_qtd} (now) + {HHHH11_buy} (buy) \t | + R$ {HHHH11_amount:.2f} | R$ {HHHH11_tot_amount:.2f}")
    print(f"IIII11: {IIII11_total} = {IIII11_min_qtd} (now) + {IIII11_buy} (buy) \t | + R$ {IIII11_amount:.2f} | R$ {IIII11_tot_amount:.2f}")
    
    print('-' * 20)
    total = AAAA11_prc * AAAA11.solution_value() + \
            BBBB11_prc * BBBB11.solution_value() + \
            CCCC11_prc * CCCC11.solution_value() + \
            DDDD11_prc * DDDD11.solution_value() + \
            EEEE11_prc * EEEE11.solution_value() + \
            FFFF11_prc * FFFF11.solution_value() + \
            GGGG11_prc * GGGG11.solution_value() + \
            HHHH11_prc * HHHH11.solution_value() + \
            IIII11_prc * IIII11.solution_value()
    print(f"Valor de proventos estimado maximidado: R$ {solver.Objective().Value():.2f}")
    print(f"Total investido neste cenário: R$ {total:.2f}")
    print(f"Yield total neste cenário: {(solver.Objective().Value() / total)*100:.2f}%")

else:
    print(f"O Solver não conseguiu encontrar uma solução ótima.") 

Solução:
Calculado em 43.00ms utilizando 0 iterações
--------------------
AAAA11: 15.0 = 0 (now) + 15.0 (buy) 	 | + R$ 1127.25 | R$ 1127.25
BBBB11: 7.0 = 0 (now) + 7.0 (buy) 	 | + R$ 1056.30 | R$ 1056.30
CCCC11: 11.0 = 0 (now) + 11.0 (buy) 	 | + R$ 1108.36 | R$ 1108.36
DDDD11: 8.0 = 0 (now) + 8.0 (buy) 	 | + R$ 1166.72 | R$ 1166.72
EEEE11: 24.0 = 0 (now) + 24.0 (buy) 	 | + R$ 1074.00 | R$ 1074.00
FFFF11: 12.0 = 0 (now) + 12.0 (buy) 	 | + R$ 1083.72 | R$ 1083.72
GGGG11: 11.0 = 0 (now) + 11.0 (buy) 	 | + R$ 1101.32 | R$ 1101.32
HHHH11: 17.0 = 0 (now) + 17.0 (buy) 	 | + R$ 1150.90 | R$ 1150.90
IIII11: 12.0 = 0 (now) + 12.0 (buy) 	 | + R$ 1114.68 | R$ 1114.68
--------------------
Valor de proventos estimado maximidado: R$ 99.59
Total investido neste cenário: R$ 9983.25
Yield total neste cenário: 1.00%


# END